# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from pinn import PINN

In [ ]:
in_filepath = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"

wing_df = pd.read_csv(in_filepath + points_filename)

print(wing_df)

In [ ]:
x_min, y_min, z_min = wing_df.min()
x_max, y_max, z_max = wing_df.max()

overall_min = min(x_min, y_min, z_min)
overall_max = max(x_max, y_max, z_max)

range = (overall_min, overall_max)
print(range)

In [ ]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter_3d(wing_df, x='x', y='y', z='z')
fig.update_traces(marker_size = 1)
fig.update_traces(marker_color = 'purple')
fig.update_layout(
    scene = dict(
        xaxis = dict(range=range),
        yaxis = dict(range=range),
        zaxis = dict(range=range)))

fig.show()

In [ ]:
out_filepath = "/Users/ggito/repos/pinns/data/"
pinn = torch.load(out_filepath + 'pinn9.pt')
pinn.eval()

In [ ]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 20
Ny = 20
Nz = 20
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.full(Nt, 0)

x_grid, y_grid, z_grid = np.meshgrid(x_test, y_test, z_test)

In [ ]:
t_test

In [ ]:
xyz_combinations = torch.cartesian_prod(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32), torch.tensor(z_test, dtype=torch.float32))

In [ ]:
num_of_points = xyz_combinations.shape[0]

In [ ]:
t = torch.full((num_of_points, 1), 0.1)

In [ ]:
xyzt_combinations = torch.cat((xyz_combinations, t), dim=1)

In [ ]:
xyzt_combinations.shape

In [ ]:
if torch.backends.mps.is_available():
  device = torch.device("mps")
  x = torch.ones(1, device=device)
  print(x)
else:
  print("MPS device not found.")
  device = "cpu"

In [ ]:
input = xyzt_combinations.to(device)

In [ ]:
output = pinn(input)

In [ ]:
output.shape

In [ ]:
u_test = output[:, 0].to("cpu").detach().numpy()
v_test = output[:, 1].to("cpu").detach().numpy()
w_test = output[:, 2].to("cpu").detach().numpy()
p_test = output[:, 3].to("cpu").detach().numpy()

In [ ]:
velocity_norm = np.linalg.norm([u_test, v_test, w_test], axis=0)

In [ ]:
velocity_norm.min()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

scatter = go.Scatter3d(
    x=wing_df['x'],
    y=wing_df['y'],
    z=wing_df['z'],
    mode='markers',
    marker=dict(size=1, color='purple')
)

cone = go.Cone(
    x=x_grid.flatten(),
    y=y_grid.flatten(),
    z=z_grid.flatten(),
    u=u_test,
    v=v_test,
    w=w_test,
    colorscale='Bluered',
    cmin=velocity_norm.min(),
    cmax=velocity_norm.max(),
    colorbar_title='Velocity<br>Magnitude',
    sizemode="absolute",
    sizeref=0.2
)

fig = go.Figure(data=[scatter, cone])

fig.update_layout(
    scene = dict(
        xaxis = dict(range=range),
        yaxis = dict(range=range),
        zaxis = dict(range=range)))

fig.show()